<a href="https://colab.research.google.com/github/manjeet704/Datascience/blob/prac_code/Tranformer_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformes

In [3]:
from transformers import pipeline

In [ ]:
Classifier=pipeline('sentiment-analysis')

In [5]:
Classifier('We are very happy to show you the transformer library')

[{'label': 'POSITIVE', 'score': 0.9998239874839783}]

In [6]:
results=Classifier(["The food is not good","The food is good"])
for result in results:
  print(f"label: {result['label']}, with score: {round(result['score'],4)}")

label: NEGATIVE, with score: 0.9997
label: POSITIVE, with score: 0.9999


In [8]:
classifier = pipeline('sentiment-analysis', model = 'nlptown/bert-base-multilingual-uncased-sentiment')

In [9]:
classifier('The food is good')

[{'label': '4 stars', 'score': 0.4961555004119873}]

In [10]:
classifier('Das Essen ist gut') # German language

[{'label': '4 stars', 'score': 0.5022420287132263}]

In [12]:
classifier('La nourriture est bonne') # French language

[{'label': '4 stars', 'score': 0.4808807075023651}]

In [13]:
#same thing to do in local
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt = True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model = model_name, tokenizer = tokenizer)

In [ ]:
classifier('the movie was great')

In [ ]:
classifier('Food is awesome')

In [17]:
classifier('In Cricket team Batsman are good , but the Bowler doing not very well')

[{'label': '3 stars', 'score': 0.6007073521614075}]